In [52]:
import networkx as nx
import numpy as np
import Visualizer

In [77]:
G = nx.gnm_random_graph(2000, 10000, seed=42)

In [45]:
char_path_length = 'not connected'
pos_max = np.argmax([len(listElem) for listElem in sorted(nx.connected_components(G))])
char_path_length_component = [nx.average_shortest_path_length(G.subgraph(g)) for g in sorted(nx.connected_components(G))][pos_max]

density = nx.density(G)

transitivity = nx.transitivity(G)
clustering_coef = nx.average_clustering(G)

n_connected_comp = nx.number_connected_components(G)
size_connected_comp = sum([len(listElem) for listElem in sorted(nx.connected_components(G))])/len(sorted(nx.connected_components(G)))

print(f'original characteristic path length {char_path_length}, \noriginal characteristic path length of biggest component {char_path_length_component},\noriginal network density {density},\noriginal transitivity {transitivity}, \noriginal average clustering coefficient {clustering_coef}, \noriginal number of connected components {n_connected_comp}, \noriginal average size of a connected component {size_connected_comp}')

original characteristic path length not connected, 
original characteristic path length of biggest component 5.639364329929552,
original network density 0.00282326369282891,
original transitivity 0.0035790980672870437, 
original average clustering coefficient 0.003164127970028592, 
original number of connected components 37, 
original average size of a connected component 34.810810810810814


In [93]:
def simulate_sirs(G,beta,gamma,xi,f,lst_initial_immune,lst_initial_infected):
    attDict = {}
    for node in lst_initial_immune:
        attDict[node] = [0]
    for node in lst_initial_infected:
        attDict[node] = [1]
    t = 0
    D = []
    D.append([len(lst_initial_immune), len(lst_initial_infected), 0])
    I = 645
    #Uncomment when creating newGraph:
    while t < f #and I < 644:
        S = 0
        I = 0
        R = 0
        for node in list(attDict.keys()):
            if attDict[node][t] == 0:
                if np.random.random() < beta:
                    attDict[node].append(1)
                    I += 1
                else:
                    attDict[node].append(0)
                    S += 1
            if attDict[node][t] == 1:
                if np.random.random() < gamma:
                    attDict[node].append(2)
                    R += 1
                else:
                    attDict[node].append(1)
                    I += 1
            if attDict[node][t] == 2:
                if np.random.random() < xi:
                    attDict[node].append(0)
                    S += 1
                else:
                    attDict[node].append(2)
                    R += 1
        D.append([S,I,R])
        t+=1
    nx.set_node_attributes(G,attDict,"state")
    infected = []
    for node in G.nodes():
        if G.nodes[node]["state"][t-1] == 1:
            infected.append(node)
    if len(infected) < 644:
        newGraph = G.subgraph(infected)
    else:
        newGraph = G.subgraph(np.random.choice(infected, 644, replace=False))
    return (D, newGraph)

In [94]:
sim = simulate_sirs(G, 0.3, 0.4, 0.1, 50, list(G.nodes)[:1000], list(G.nodes)[1000:])

In [95]:
D = sim[0]
ngraph = sim[1]
(len(ngraph.nodes()), len(ngraph.edges()))

(644, 1056)

In [89]:
Visualizer.showSIRS(G, "test_simulation.html", 0.3, 0.4, 0.1, 50, D)

KeyboardInterrupt: 

In [ ]:
Visualizer.showData(D, "test_data.html", 0.3, 0.4, 0.1)